In [1]:
# Imports

from mistralai import Mistral
from datetime import datetime
import pandas as pd
import random
import time


In [ ]:
api_key = "" #INSERT YOUR KEY HERE
model = "mistral-large-2407"
temperatures = [0.0, 0.7, 1.4]
dataset_name = "mistral_dataset_raw.csv"
b_df = pd.read_csv("prompts.csv")

client = Mistral(api_key=api_key)

def talk_to_mistral(temp, prompt):
    
    chat_response = client.chat.complete(
        model = model,
        temperature = temp,
        messages = [
            {
                "role": "user",
                "content": prompt,
            },
        ]
    )

    return chat_response.dict()
    

In [ ]:
# Start the dataset
first_prompt = "Greet me in 5 tokens"
temp_0 = 0.7

response_dict = talk_to_mistral(temp_0, first_prompt)
response_dict["datetime"] = datetime.now()
response_dict["hash"] = "0000"
response_dict["temperature"] = temp_0
response_dict["prompt"] = first_prompt
response_dict["request_prompt_ID"] = -1
response_dict["response_text"] = response_dict["choices"][0]["message"]["content"]
response_dict["origin"] = "Greet"
response_dict["prompt_type"] = "open_ended"

pd.json_normalize(response_dict).to_csv(dataset_name)

In [ ]:
# Generate the dataset
no_works = []
for i in range(len(b_df))[492:]:
    for temp in temperatures:

        try:
            current_df = pd.read_csv(dataset_name)
            
            prompt = b_df.iloc[i,2]
    
            response_dict = talk_to_mistral(temp, prompt)
            response_dict["datetime"] = datetime.now()
            response_dict["hash"] = b_df.iloc[i,1]
            response_dict["temperature"] = temp
            response_dict["prompt"] = prompt
            response_dict["request_prompt_ID"] = b_df.index[i]
            response_dict["response_text"] = response_dict["choices"][0]["message"]["content"]
            response_dict["origin"] = b_df.iloc[i,3]
            response_dict["prompt_type"] = b_df.iloc[i,4]
    
            # Create a row with the data
            row_df = pd.json_normalize(response_dict)
    
    
            # Merge the existing datasets
            merged_df = pd.concat([current_df, row_df], axis=0, ignore_index=True, sort=False)
    
            # Save the merged_df in the directory CSV
            merged_df.to_csv(dataset_name, index=False)
    
            # Print Done
            print(i, temp)
            print(response_dict["response_text"])
            print("\n")
            
        except:
            no_works.append((i, temp))
            print("-----------------------------")
            print(f"The following temp and request ID did not work: {i}, temp: {temp}")
            print("-----------------------------")


In [ ]:
# Generate the texts that could not be generated in the first round

no_works2 = []

for pair in no_works:

    i, temp = pair
    
    try:
        current_df = pd.read_csv(dataset_name)
        
        prompt = b_df.iloc[i,2]
    
        response_dict = talk_to_mistral(temp, prompt)
        response_dict["datetime"] = datetime.now()
        response_dict["hash"] = b_df.iloc[i,1]
        response_dict["temperature"] = temp
        response_dict["prompt"] = prompt
        response_dict["request_prompt_ID"] = b_df.index[i]
        response_dict["response_text"] = response_dict["choices"][0]["message"]["content"]
        response_dict["origin"] = b_df.iloc[i,3]
        response_dict["prompt_type"] = b_df.iloc[i,4]
    
        # Create a row with the data
        row_df = pd.json_normalize(response_dict)
    
    
        # Merge the existing datasets
        merged_df = pd.concat([current_df, row_df], axis=0, ignore_index=True, sort=False)
    
        # Save the merged_df in the directory CSV
        merged_df.to_csv(dataset_name, index=False)
    
        # Print Done
        print(i, temp)
        print(response_dict["response_text"])
        print("\n")
        
    except:
        no_works2.append((i, temp))
        print("-----------------------------")
        print(f"The following temp and request ID did not work: {i}, temp: {temp}")
        print("-----------------------------")

In [ ]:
raw_df = pd.read_csv("mistral_dataset_raw.csv")
print(len(raw_df))
raw_df.head(2)

In [ ]:
non_duplicates_df = raw_df.drop_duplicates(subset=['temperature', 'request_prompt_ID'])

In [ ]:
# Check the sizi difference
len(raw_df), len(non_duplicates_df)

In [ ]:
clean_df = non_duplicates_df[["request_prompt_ID", "hash", "temperature", "origin", "prompt_type", "prompt", "response_text"]]

clean_df = clean_df[clean_df["request_prompt_ID"] >= 0].reset_index()

clean_df = clean_df.sort_values(by=["request_prompt_ID", "temperature"]).drop("index", axis=1)

In [ ]:
clean_df.to_csv("mistral_ai_dataset.csv")

In [1]:
# explore the generated dataset
import pandas as pd
df = pd.read_csv("mistral_ai_dataset.csv")
df.columns

Index(['request_prompt_ID', 'hash', 'temperature', 'origin', 'prompt_type',
       'prompt', 'response_text'],
      dtype='object')

In [ ]:
df = df.drop(['Unnamed: 0.1'], axis=1)
df.head()

In [26]:
# The following prompt (hash: 401104c55e54c0c969e8b0fd0f0302f4) is in russian so I will delete it.
df[df["hash"] == "401104c55e54c0c969e8b0fd0f0302f4"].prompt[4731][:260]

'Given a passage, please paraphrase it. The content should be the same. The passage is:ни чего нового как правильноНичего страшного. View more. 1. И Новый год как раз и есть такой пункт. Чтобы вы могли правильно воспользоваться «остановкой», AdMe.ru расскажет, '

In [30]:
df = df[df["hash"] != "401104c55e54c0c969e8b0fd0f0302f4"]

In [3]:
df.to_csv("mistral_ai_dataset.csv", index=False)